# Predictive Modelling in Healthcare

## Problem Statement
This project aims to build machine learning models for healthcare risk prediction using structured EHR data.

## Data Preprocessing

The raw dataset contains hospital admission records of diabetic patients.

This preprocessing stage performs the following steps:

1. **Duplicate Removal**  
   Duplicate encounters are removed using `encounter_id` to ensure data integrity.

2. **Missing Value Handling**  
   - Replace '?' with NaN  
   - Drop columns with excessive missing values (`weight`, `payer_code`, `medical_specialty`)

3. **Target Variable Construction**  
   The multi-class `readmitted` column is converted into binary:
   - 1 → Readmitted within 30 days (`<30`)
   - 0 → Otherwise

4. **Institution-Based Split**  
   Instead of random row splitting, we simulate real-world hospital generalization by splitting based on `admission_source_id` (proxy for institution).

5. **One-Hot Encoding**  
   Categorical variables are encoded to numerical form.

6. **Feature Scaling**  
   Numerical features are standardized using `StandardScaler` to improve model convergence.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# 1. LOAD DATA
df = pd.read_csv("diabetic_data.csv")
print("Original Shape:", df.shape)

# 2. REMOVE DUPLICATES (by encounter)
df = df.drop_duplicates(subset=["encounter_id"])

# 3. HANDLE MISSING VALUES
df.replace("?", np.nan, inplace=True)

# Drop high-missing columns
df.drop(columns=["weight", "payer_code", "medical_specialty"], inplace=True)

# 4. TARGET VARIABLE (Binary Classification)
df["readmitted"] = df["readmitted"].apply(lambda x: 1 if x == "<30" else 0)

print("Target Distribution:\n", df["readmitted"].value_counts())

# 5. INSTITUTION-BASED SPLIT 

institution_col = "admission_source_id"

unique_institutions = df[institution_col].unique()

np.random.seed(42)

train_institutions = np.random.choice(
    unique_institutions,
    size=int(len(unique_institutions) * 0.7),
    replace=False
)

test_institutions = list(set(unique_institutions) - set(train_institutions))

train_df = df[df[institution_col].isin(train_institutions)].copy()
test_df = df[df[institution_col].isin(test_institutions)].copy()

print("Number of Train Institutions:", len(train_institutions))
print("Number of Test Institutions:", len(test_institutions))
print("Train Shape (Pre-Encode):", train_df.shape)
print("Test Shape (Pre-Encode):", test_df.shape)

# 6. FEATURE / ID HANDLING

train_df.drop(columns=["encounter_id"], inplace=True)
test_df.drop(columns=["encounter_id"], inplace=True)

# 7. ONE-HOT ENCODING
categorical_cols = train_df.select_dtypes(include=["object", "string"]).columns
train_df = pd.get_dummies(train_df, columns=categorical_cols, drop_first=True)
test_df = pd.get_dummies(test_df, columns=categorical_cols, drop_first=True)

train_df, test_df = train_df.align(test_df, join="left", axis=1, fill_value=0)

print("After Encoding Shape:")
print("Train:", train_df.shape)
print("Test:", test_df.shape)

# 8. NORMALIZE NUMERICAL FEATURES
exclude_cols = ["readmitted", "hospital_id", "patient_nbr"]

numerical_cols = [
    col for col in train_df.columns
    if col not in exclude_cols and train_df[col].dtype != "uint8"
]

scaler = StandardScaler()

train_df[numerical_cols] = scaler.fit_transform(train_df[numerical_cols])
test_df[numerical_cols] = scaler.transform(test_df[numerical_cols])

# 9. SAVE PROCESSED DATA
train_df.to_csv("train_processed.csv", index=False)
test_df.to_csv("test_processed.csv", index=False)

print("Preprocessing Completed Successfully ✅")
print("Final Train Shape:", train_df.shape)
print("Final Test Shape:", test_df.shape)

## Feature Engineering & Class Imbalance Handling

Feature engineering enhances predictive power by creating clinically meaningful variables.

### Engineered Features:

- **Admission Frequency**  
  Counts how many times a patient appears in the dataset (proxy for chronic severity).

- **Medication Intensity**  
  Counts number of active medication-related indicators (e.g., insulin, metformin).

### Handling Class Imbalance

Hospital readmission is typically imbalanced.

We apply **SMOTE (Synthetic Minority Oversampling Technique)** to:
- Increase minority class representation
- Improve recall performance
- Reduce bias toward majority class

SMOTE is applied only on training data to prevent data leakage.

In [ ]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE

# 1. LOAD CLEANED (PRE-ENCODED) DATA
train_df = pd.read_csv("train_processed.csv")
test_df = pd.read_csv("test_processed.csv")

print("Initial Train Shape:", train_df.shape)
print("Initial Class Distribution:")
print(train_df["readmitted"].value_counts())

# 2. FEATURE ENGINEERING
# A. Admission Frequency

if "patient_nbr" in train_df.columns:
    admission_counts = train_df.groupby("patient_nbr").size()
    train_df["admission_frequency"] = train_df["patient_nbr"].map(admission_counts)

    test_df["admission_frequency"] = test_df["patient_nbr"].map(admission_counts)
    test_df["admission_frequency"].fillna(1, inplace=True)

# B. Medication Intensity
med_cols = [col for col in train_df.columns if "metformin" in col or "insulin" in col]

if len(med_cols) > 0:
    train_df["medication_count"] = (train_df[med_cols] > 0).sum(axis=1)
    test_df["medication_count"] = (test_df[med_cols] > 0).sum(axis=1)

print("After Feature Engineering:", train_df.shape)

# 3. REMOVE NON-MODELING COLUMNS
drop_cols = ["patient_nbr"]  

for col in drop_cols:
    if col in train_df.columns:
        train_df.drop(columns=[col], inplace=True)
        test_df.drop(columns=[col], inplace=True)

# 4. SPLIT FEATURES & TARGET
X_train = train_df.drop("readmitted", axis=1)
y_train = train_df["readmitted"]

X_test = test_df.drop("readmitted", axis=1)
y_test = test_df["readmitted"]

# 5. APPLY SMOTE (TRAIN ONLY)
smote = SMOTE(
    sampling_strategy=0.5,  
    random_state=42
)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

print("\nAfter SMOTE:")
print(pd.Series(y_train_balanced).value_counts())
print("Balanced Train Shape:", X_train_balanced.shape)

# 6. SAVE FINAL DATA
pd.DataFrame(X_train_balanced).to_csv("X_train_final.csv", index=False)
pd.DataFrame(X_test).to_csv("X_test_final.csv", index=False)
pd.Series(y_train_balanced).to_csv("y_train_final.csv", index=False)
pd.Series(y_test).to_csv("y_test_final.csv", index=False)

print("\nFeature Engineering + SMOTE Completed Successfully ✅")

## Model Training & Comparison

We train and compare three machine learning models:

1. **Logistic Regression**
   - Baseline linear classifier
   - Fast and interpretable

2. **Random Forest**
   - Ensemble of decision trees
   - Handles non-linearity and feature interactions

3. **XGBoost**
   - Gradient boosting algorithm
   - Optimized for performance and scalability

Each model is evaluated using:

- Accuracy
- Precision
- Recall
- F1 Score
- ROC-AUC
- Training Time

The best model is selected based on ROC-AUC.

In [ ]:
import pandas as pd
import numpy as np
import time
import warnings
warnings.filterwarnings("ignore")

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import (
    roc_auc_score, accuracy_score,
    precision_score, recall_score, f1_score
)

# LOAD DATA
X_train = pd.read_csv("X_train_final.csv").values
X_test  = pd.read_csv("X_test_final.csv").values
y_train = pd.read_csv("y_train_final.csv").values.ravel()
y_test  = pd.read_csv("y_test_final.csv").values.ravel()

X_train = np.nan_to_num(X_train)
X_test  = np.nan_to_num(X_test)

print("Data Loaded ✅")
print("Train Shape:", X_train.shape)
print("Test Shape :", X_test.shape)
print()

# HELPER — train, predict, time, score
def evaluate_model(name, model, X_tr, y_tr, X_te, y_te, threshold=0.5):

    start = time.time()
    model.fit(X_tr, y_tr)
    train_time = round(time.time() - start, 2)

    y_prob = model.predict_proba(X_te)[:, 1]
    y_pred = (y_prob > threshold).astype(int)

    return {
        "Model"             : name,
        "Training Time (s)" : train_time,
        "Accuracy"          : round(accuracy_score(y_te, y_pred),  4),
        "Precision"         : round(precision_score(y_te, y_pred, zero_division=0), 4),
        "Recall"            : round(recall_score(y_te, y_pred),    4),
        "F1"                : round(f1_score(y_te, y_pred),        4),
        "ROC-AUC"           : round(roc_auc_score(y_te, y_prob),   4),
    }, model

# MODEL 1 — Logistic Regression (sklearn baseline)
print("Training Logistic Regression (sklearn)...")
lr = LogisticRegression(
    max_iter=500,
    solver="saga",          
    C=1.0,
    n_jobs=-1,
    random_state=42
)
lr_results, lr_model = evaluate_model(
    "Logistic Regression (sklearn)", lr, X_train, y_train, X_test, y_test
)
print("Done ✅  AUC:", lr_results["ROC-AUC"],
      "| Time:", lr_results["Training Time (s)"], "s\n")

# MODEL 2 — Random Forest
print("Training Random Forest...")
rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    class_weight="balanced",   
    n_jobs=-1,
    random_state=42
)
rf_results, rf_model = evaluate_model(
    "Random Forest", rf, X_train, y_train, X_test, y_test
)
print("Done ✅  AUC:", rf_results["ROC-AUC"],
      "| Time:", rf_results["Training Time (s)"], "s\n")

# MODEL 3 — XGBoost
print("Training XGBoost...")
xgb = XGBClassifier(
    n_estimators=50,
    max_depth=4,
    learning_rate=0.1,
    eval_metric="logloss",
    n_jobs=-1,
    random_state=42
)
xgb_results, xgb_model = evaluate_model(
    "XGBoost", xgb, X_train, y_train, X_test, y_test
)
print("Done ✅  AUC:", xgb_results["ROC-AUC"],
      "| Time:", xgb_results["Training Time (s)"], "s\n")

# COMPARISON TABLE
results_df = pd.DataFrame([lr_results, rf_results, xgb_results])

print("\n" + "=" * 70)
print("              PHASE 4 — MODEL COMPARISON SUMMARY")
print("=" * 70)
print(results_df.to_string(index=False))
print("=" * 70)

spark_lr_auc  = None  
spark_lr_time = None  

if spark_lr_auc is not None:
    print("\n--- Distributed Comparison (PySpark) ---")
    print(f"Spark Logistic Regression | AUC: {spark_lr_auc} | "
          f"Training Time: {spark_lr_time} s")
    print("sklearn Logistic Regression | AUC:", lr_results["ROC-AUC"],
          "| Training Time:", lr_results["Training Time (s)"], "s")

# BEST MODEL
best = results_df.loc[results_df["ROC-AUC"].idxmax(), "Model"]
print(f"\n🏆 Best Model by ROC-AUC: {best}")
print("\nPhase 4 Completed ✅")

results_df.to_csv("phase4_comparison.csv", index=False)
print("Comparison saved → phase4_comparison.csv")


## Model Evaluation & Threshold Optimization

After training, the best model (XGBoost) is evaluated using:

### 1. ROC Curve
Measures discrimination ability across thresholds.

### 2. Precision-Recall Curve
More informative for imbalanced datasets.

### 3. Threshold Tuning
Instead of using default 0.5 threshold, multiple thresholds are tested.
The threshold with highest F1-score is selected.

### 4. Institution-wise Evaluation
Performance is evaluated per institution to measure generalization gap across hospitals.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import (
    roc_curve,
    roc_auc_score,
    precision_recall_curve,
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)
from xgboost import XGBClassifier

# =====================================================
# LOAD DATA
# =====================================================
X_train = pd.read_csv("X_train_final.csv").values
X_test = pd.read_csv("X_test_final.csv").values
y_train = pd.read_csv("y_train_final.csv").values.ravel()
y_test = pd.read_csv("y_test_final.csv").values.ravel()

# Remove any NaNs (safety)
X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

# =====================================================
# TRAIN BEST MODEL (XGBoost)
# =====================================================
xgb = XGBClassifier(
    n_estimators=50,
    max_depth=4,
    learning_rate=0.1,
    eval_metric="logloss",
    n_jobs=-1
)

print("\nTraining XGBoost for Phase 5...")
xgb.fit(X_train, y_train)

# =====================================================
# PREDICT PROBABILITIES
# =====================================================
y_prob = xgb.predict_proba(X_test)[:, 1]

# =====================================================
# 1️⃣ ROC CURVE
# =====================================================
fpr, tpr, roc_thresholds = roc_curve(y_test, y_prob)
auc_score = roc_auc_score(y_test, y_prob)

plt.figure()
plt.plot(fpr, tpr)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title(f"ROC Curve (AUC = {auc_score:.4f})")
plt.show()

print("ROC-AUC:", round(auc_score, 4))

# =====================================================
# 2️⃣ PRECISION-RECALL CURVE
# =====================================================
precision_vals, recall_vals, pr_thresholds = precision_recall_curve(y_test, y_prob)

plt.figure()
plt.plot(recall_vals, precision_vals)
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve")
plt.show()

# =====================================================
# 3️⃣ THRESHOLD TUNING
# =====================================================
thresholds_to_test = [0.5, 0.4, 0.3, 0.25, 0.2, 0.15, 0.1]

print("\n================ THRESHOLD ANALYSIS ================\n")

best_f1 = 0
best_threshold = 0.5

for t in thresholds_to_test:
    y_pred = (y_prob > t).astype(int)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    print(f"Threshold = {t}")
    print("Accuracy:", round(acc, 4))
    print("Precision:", round(prec, 4))
    print("Recall:", round(rec, 4))
    print("F1:", round(f1, 4))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("--------------------------------------------------")

    if f1 > best_f1:
        best_f1 = f1
        best_threshold = t

print("\nBest Threshold Based on F1:", best_threshold)

# =====================================================
# 4️⃣ FINAL METRICS AT BEST THRESHOLD
# =====================================================
y_final = (y_prob > best_threshold).astype(int)

print("\n================ FINAL PERFORMANCE ================\n")
print("Accuracy:", round(accuracy_score(y_test, y_final), 4))
print("Precision:", round(precision_score(y_test, y_final), 4))
print("Recall:", round(recall_score(y_test, y_final), 4))
print("F1:", round(f1_score(y_test, y_final), 4))
print("ROC-AUC:", round(auc_score, 4))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_final))

# =====================================================
# 5️⃣ INSTITUTION-WISE GENERALIZATION GAP
# =====================================================
print("\n================ INSTITUTION-WISE PERFORMANCE ================\n")

# Reload test data WITH institution column
test_full = pd.read_csv("test_processed.csv")

# Align predictions
test_full = test_full.reset_index(drop=True)
test_full["prob"] = y_prob
test_full["prediction"] = y_final

institution_col = "admission_source_id"  # proxy institution

institution_results = []

for inst in test_full[institution_col].unique():
    sub = test_full[test_full[institution_col] == inst]

    if len(sub) < 50:   # ignore very small groups
        continue

    inst_auc = roc_auc_score(sub["readmitted"], sub["prob"])
    inst_recall = recall_score(sub["readmitted"], sub["prediction"])

    institution_results.append({
        "Institution": inst,
        "Samples": len(sub),
        "AUC": round(inst_auc, 4),
        "Recall": round(inst_recall, 4)
    })

institution_df = pd.DataFrame(institution_results)

print(institution_df)

print("\nAUC Mean Across Institutions:",
      round(institution_df["AUC"].mean(), 4))

print("AUC Std Dev Across Institutions:",
      round(institution_df["AUC"].std(), 4))

## Scalability Analysis

To evaluate computational scalability, the model is trained on increasing fractions of training data:

- 20%
- 40%
- 60%
- 80%
- 100%

For each fraction:
- Training time is recorded
- ROC-AUC is computed

This experiment demonstrates:
- Computational growth behavior
- Performance stability with increasing data size

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import (
    roc_curve,
    roc_auc_score,
    precision_recall_curve,
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)
from xgboost import XGBClassifier

# LOAD DATA
X_train = pd.read_csv("X_train_final.csv").values
X_test = pd.read_csv("X_test_final.csv").values
y_train = pd.read_csv("y_train_final.csv").values.ravel()
y_test = pd.read_csv("y_test_final.csv").values.ravel()

X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

# TRAIN BEST MODEL (XGBoost)
xgb = XGBClassifier(
    n_estimators=50,
    max_depth=4,
    learning_rate=0.1,
    eval_metric="logloss",
    n_jobs=-1
)

print("\nTraining XGBoost for Phase 5...")
xgb.fit(X_train, y_train)

# PREDICT PROBABILITIES
y_prob = xgb.predict_proba(X_test)[:, 1]

# 1️⃣ ROC CURVE
fpr, tpr, roc_thresholds = roc_curve(y_test, y_prob)
auc_score = roc_auc_score(y_test, y_prob)

plt.figure()
plt.plot(fpr, tpr)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title(f"ROC Curve (AUC = {auc_score:.4f})")
plt.show()

print("ROC-AUC:", round(auc_score, 4))

# 2️⃣ PRECISION-RECALL CURVE
precision_vals, recall_vals, pr_thresholds = precision_recall_curve(y_test, y_prob)

plt.figure()
plt.plot(recall_vals, precision_vals)
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve")
plt.show()

# 3️⃣ THRESHOLD TUNING
thresholds_to_test = [0.5, 0.4, 0.3, 0.25, 0.2, 0.15, 0.1]

print("\n================ THRESHOLD ANALYSIS ================\n")

best_f1 = 0
best_threshold = 0.5

for t in thresholds_to_test:
    y_pred = (y_prob > t).astype(int)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    print(f"Threshold = {t}")
    print("Accuracy:", round(acc, 4))
    print("Precision:", round(prec, 4))
    print("Recall:", round(rec, 4))
    print("F1:", round(f1, 4))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("--------------------------------------------------")

    if f1 > best_f1:
        best_f1 = f1
        best_threshold = t

print("\nBest Threshold Based on F1:", best_threshold)

# 4️⃣ FINAL METRICS AT BEST THRESHOLD
y_final = (y_prob > best_threshold).astype(int)

print("\n================ FINAL PERFORMANCE ================\n")
print("Accuracy:", round(accuracy_score(y_test, y_final), 4))
print("Precision:", round(precision_score(y_test, y_final), 4))
print("Recall:", round(recall_score(y_test, y_final), 4))
print("F1:", round(f1_score(y_test, y_final), 4))
print("ROC-AUC:", round(auc_score, 4))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_final))

# 5️⃣ INSTITUTION-WISE GENERALIZATION GAP
print("\n================ INSTITUTION-WISE PERFORMANCE ================\n")

test_full = pd.read_csv("test_processed.csv")

test_full = test_full.reset_index(drop=True)
test_full["prob"] = y_prob
test_full["prediction"] = y_final

institution_col = "admission_source_id"  

institution_results = []

for inst in test_full[institution_col].unique():
    sub = test_full[test_full[institution_col] == inst]

    if len(sub) < 50:   
        continue

    inst_auc = roc_auc_score(sub["readmitted"], sub["prob"])
    inst_recall = recall_score(sub["readmitted"], sub["prediction"])

    institution_results.append({
        "Institution": inst,
        "Samples": len(sub),
        "AUC": round(inst_auc, 4),
        "Recall": round(inst_recall, 4)
    })

institution_df = pd.DataFrame(institution_results)

print(institution_df)

print("\nAUC Mean Across Institutions:",
      round(institution_df["AUC"].mean(), 4))

print("AUC Std Dev Across Institutions:",
      round(institution_df["AUC"].std(), 4))

## 7️⃣ Model Explainability & Interpretability

In healthcare applications, model performance alone is not sufficient.  
Clinicians must understand *why* a model makes a prediction.

To ensure interpretability, we analyze:

### 1️⃣ Feature Importance
Using XGBoost’s built-in feature importance scores, we identify which variables most influence readmission prediction.

This helps:
- Understand key clinical drivers
- Validate medical plausibility
- Increase trust in the model

### 2️⃣ Scalability vs Performance Trade-off
We evaluate how model performance (ROC-AUC) changes as training data increases.

This ensures:
- The model scales efficiently
- Performance stabilizes with more data
- Computational growth is reasonable

Interpretability and scalability together ensure the model is:
- Transparent
- Clinically meaningful
- Deployable in real-world hospital systems

In [ ]:
import pandas as pd
import numpy as np
import shap
import matplotlib.pyplot as plt
from xgboost import XGBClassifier

# LOAD DATA WITH SAFE FEATURE NAMES
X_train = pd.read_csv("X_train_final.csv")
X_test = pd.read_csv("X_test_final.csv")
y_train = pd.read_csv("y_train_final.csv").values.ravel()

# Replace illegal characters in column names
X_train.columns = (
    X_train.columns
    .str.replace("[", "_", regex=False)
    .str.replace("]", "_", regex=False)
    .str.replace("<", "lt_", regex=False)
    .str.replace(">", "gt_", regex=False)
    .str.replace(" ", "_", regex=False)
)

X_test.columns = X_train.columns

# Fill NaNs safely
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

print("Train Shape:", X_train.shape)

# TRAIN XGBOOST (Same config as Phase 5)
xgb = XGBClassifier(
    n_estimators=50,
    max_depth=4,
    learning_rate=0.1,
    eval_metric="logloss",
    n_jobs=-1
)

print("\nTraining model for SHAP...")
xgb.fit(X_train, y_train)

# CREATE SHAP EXPLAINER
explainer = shap.TreeExplainer(xgb)

sample_size = min(2000, len(X_test))
X_sample = X_test.sample(sample_size, random_state=42)

print("Generating SHAP values...")
shap_values = explainer.shap_values(X_sample)

# 1️⃣ GLOBAL FEATURE IMPORTANCE (BAR)
plt.figure()
shap.summary_plot(
    shap_values,
    X_sample,
    plot_type="bar",
    show=False
)
plt.title("Global Feature Importance (SHAP)")
plt.show()

# 2️⃣ DETAILED SHAP SUMMARY (Impact Distribution)
plt.figure()
shap.summary_plot(
    shap_values,
    X_sample,
    show=False
)
plt.title("SHAP Summary Plot")
plt.show()

# 3️⃣ TOP 10 IMPORTANT FEATURES (Numerical Output)
mean_abs_shap = np.abs(shap_values).mean(axis=0)
feature_importance = pd.DataFrame({
    "Feature": X_sample.columns,
    "Mean |SHAP|": mean_abs_shap
}).sort_values(by="Mean |SHAP|", ascending=False)

print("\nTop 10 Important Features:\n")
print(feature_importance.head(10))

# 4️⃣ INDIVIDUAL PATIENT EXPLANATION
patient_index = 0

print("\nGenerating explanation for one patient...")

shap.force_plot(
    explainer.expected_value,
    shap_values[patient_index],
    X_sample.iloc[patient_index],
    matplotlib=True
)

print("\nSHAP Analysis Completed Successfully ✅")

## Scalability Analysis

To evaluate computational scalability, the model is trained on increasing fractions of training data:

- 20%
- 40%
- 60%
- 80%
- 100%

For each fraction:
- Training time is recorded
- ROC-AUC is computed

This experiment demonstrates:
- Computational growth behavior
- Performance stability with increasing data size

In [ ]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier

# LOAD DATA
X_train = pd.read_csv("X_train_final.csv").values
X_test = pd.read_csv("X_test_final.csv").values
y_train = pd.read_csv("y_train_final.csv").values.ravel()
y_test = pd.read_csv("y_test_final.csv").values.ravel()

# Remove NaNs
X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

# SHUFFLE TRAINING DATA (IMPORTANT)
np.random.seed(42)
indices = np.random.permutation(len(X_train))
X_train = X_train[indices]
y_train = y_train[indices]

# DATA FRACTIONS
fractions = [0.2, 0.4, 0.6, 0.8, 1.0]

training_times = []
auc_scores = []
records = []

print("Starting Scalability Experiment...\n")

for frac in fractions:

    size = int(len(X_train) * frac)

    X_subset = X_train[:size]
    y_subset = y_train[:size]

    model = XGBClassifier(
        n_estimators=50,     
        max_depth=4,
        learning_rate=0.1,
        eval_metric="logloss",
        n_jobs=-1
    )

    start = time.time()
    model.fit(X_subset, y_subset)
    end = time.time()

    y_prob = model.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, y_prob)

    training_time = end - start

    training_times.append(training_time)
    auc_scores.append(auc)

    records.append({
        "Data Fraction (%)": int(frac * 100),
        "Training Samples": size,
        "Training Time (sec)": round(training_time, 2),
        "ROC-AUC": round(auc, 4)
    })

    print(f"Data Fraction: {int(frac*100)}%")
    print("Training Samples:", size)
    print("Training Time:", round(training_time, 2), "sec")
    print("ROC-AUC:", round(auc, 4))
    print("--------------------------------------")

# SUMMARY TABLE
scalability_df = pd.DataFrame(records)

print("\n================ SCALABILITY SUMMARY ================\n")
print(scalability_df.to_string(index=False))

# PLOT 1 — Training Time
plt.figure()
plt.plot([f * 100 for f in fractions], training_times)
plt.xlabel("Training Data Size (%)")
plt.ylabel("Training Time (sec)")
plt.title("Scalability: Data Size vs Training Time")
plt.show()

# PLOT 2 — AUC Performance
plt.figure()
plt.plot([f * 100 for f in fractions], auc_scores)
plt.xlabel("Training Data Size (%)")
plt.ylabel("ROC-AUC")
plt.title("Scalability: Data Size vs ROC-AUC")
plt.show()

print("\nScalability Analysis Completed ✅")

## Fairness Analysis

Healthcare models must ensure equitable performance across demographic groups.

We evaluate fairness across:

- Gender
- Race

For each group, we compute:
- Recall
- Precision
- ROC-AUC

This helps identify:
- Potential bias
- Performance disparity across subpopulations

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import recall_score, precision_score, roc_auc_score
from xgboost import XGBClassifier

# LOAD ORIGINAL DATA
df = pd.read_csv("diabetic_data.csv")
df.replace("?", np.nan, inplace=True)
df["readmitted"] = df["readmitted"].apply(lambda x: 1 if x == "<30" else 0)

# RECREATE SAME INSTITUTION SPLIT (Phase 2 logic)
institution_col = "admission_source_id"

unique_institutions = df[institution_col].unique()

np.random.seed(42)
train_institutions = np.random.choice(
    unique_institutions,
    size=int(len(unique_institutions) * 0.7),
    replace=False
)

test_df = df[~df[institution_col].isin(train_institutions)].copy()
test_df = test_df.reset_index(drop=True)

# LOAD TRAINED DATA
X_train = pd.read_csv("X_train_final.csv").values
y_train = pd.read_csv("y_train_final.csv").values.ravel()
X_test = pd.read_csv("X_test_final.csv").values
y_test = pd.read_csv("y_test_final.csv").values.ravel()

X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

# TRAIN MODEL (same config as Phase 5)
model = XGBClassifier(
    n_estimators=50,
    max_depth=4,
    learning_rate=0.1,
    eval_metric="logloss",
    n_jobs=-1
)

model.fit(X_train, y_train)

y_prob = model.predict_proba(X_test)[:, 1]
best_threshold = 0.2
y_pred = (y_prob > best_threshold).astype(int)

# Attach predictions
test_df["prediction"] = y_pred
test_df["prob"] = y_prob

# FAIRNESS FUNCTION
def evaluate_group(df, column):

    print(f"\n===== Fairness: {column} =====\n")

    for group in df[column].dropna().unique():

        sub = df[df[column] == group]

        if len(sub) < 50:
            continue

        recall = recall_score(sub["readmitted"], sub["prediction"])
        precision = precision_score(sub["readmitted"], sub["prediction"], zero_division=0)
        auc = roc_auc_score(sub["readmitted"], sub["prob"])

        print(f"{column}: {group}")
        print("Samples:", len(sub))
        print("Recall:", round(recall, 4))
        print("Precision:", round(precision, 4))
        print("ROC-AUC:", round(auc, 4))
        print("-----------------------------------")


evaluate_group(test_df, "gender")
evaluate_group(test_df, "race")

print("\nFairness Analysis Completed ✅")

## Conclusion

This project demonstrates a scalable, generalizable, and fairness-aware approach to hospital readmission prediction.

Key Contributions:

- Institution-based train-test split to simulate real-world deployment
- Feature engineering for improved clinical representation
- SMOTE for imbalance handling
- Multi-model comparison
- Threshold tuning for recall optimization
- Scalability and fairness evaluation

The final XGBoost model achieved strong ROC-AUC performance while maintaining balanced subgroup performance.

This framework can be extended to real-world healthcare risk stratification systems.